# **Tile Stitching : Merging unmixed files to OME-TIFF**

The field of digital pathology often involves the management and processing of large, high-resolution images, particularly those in the form of tiled image sets. For instance, in whole-slide imaging (WSI), an entire microscope slide is scanned at high resolution, often creating gigapixel images. To manage the size of these images, they are divided into smaller, overlapping tiles. To analyze the whole slide image, these tiles need to be accurately stitched back together.

To facilitate this, we have developed a Python class, `TileStitcher`, that leverages the power of QuPath, an open-source software platform for bioimage analysis, and the OpenJDK Java development kit. This class provides an efficient way to stitch together tiled images.

The `TileStitcher` class is a Python adaptation of a Groovy script available [here](https://gist.github.com/petebankhead/b5a86caa333de1fdcff6bdee72a20abe). The original Groovy script extracts the locations of the tiles from the baseline TIFF tags, stitches them together, and writes the stitched image as a pyramidal OME-TIFF. By reimplementing this in Python, we aim to provide a more accessible and easy-to-use tool for the digital pathology community.

In this tutorial, we will walk you through the process of using the `TileStitcher` class to collect, parse, stitch, and write large sets of tiled TIFF images. This class is particularly useful for dealing with tiled images from digital pathology experiments, where each image represents a small part of a larger whole. Let's get started!


## Prerequisites

Before using the `TileStitcher` class, we need to install the necessary software and configure the environment. 

### Software Installation

The `TileStitcher` class requires QuPath and OpenJDK. Here is how to install them:

1. Download and install QuPath from its [GitHub release page](https://github.com/qupath/qupath/releases). Here we are using version 0.3.1.

```bash
wget https://github.com/qupath/qupath/releases/download/v0.3.1/QuPath-0.3.1-Linux.tar.xz
tar -xvf QuPath-0.3.1-Linux.tar.xz
```

2. Download and Install OpenJDK 17

```bash
wget https://download.oracle.com/java/17/latest/jdk-17_linux-x64_bin.deb
sudo apt install ./jdk-17_linux-x64_bin.deb


### Environment Configuration

To use `TileStitcher`, we need to set the correct paths to the QuPath library and OpenJDK. For this, we need to add the paths to the environment variables `JAVA_HOME`, `CLASSPATH`, and `LD_LIBRARY_PATH`.

The `JAVA_HOME` environment variable should be set to the path where the JDK is installed.
The `CLASSPATH` environment variable should include paths to all the QuPath library jar files.
In the initialization of TileStitcher, these environment variables are used to start the Java Virtual Machine (JVM) and import the necessary QuPath classes.

## Best Practices and Considerations for Using the TileStitcher Module

### 1. JVM Session Management

The TileStitcher module utilizes jpype to manage the JVM sessions, a departure from the python-javabridge used in other parts of the package. This difference can cause conflicts when trying to run modules concurrently within the same Python environment. Hence, it is advisable to avoid running TileStitcher operations in the same notebook where python-javabridge dependent modules are running.

### 2. Restarting Kernel to Re-initialize JVM

The jpype does not allow the JVM to be restarted within the same Python session once it has been terminated. As a result, to run the TileStitcher module again or to switch back to modules that use python-javabridge, a kernel restart might be necessary.

### 3. Segregating Workflows

To mitigate potential conflicts, consider segregating workflows based on the JVM management package they depend on. It is recommended to use separate notebooks or scripts for operations involving TileStitcher and for those involving modules that are dependent on python-javabridge.



### Initialization

Once the environment is set up, we can initialize TileStitcher.

In [1]:
import glob
import os
from pathml.preprocessing.tilestitcher import TileStitcher

# Set the path to the JDK
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-17/"
qupath_jars = glob.glob(os.path.abspath("../../tools/QuPath/lib/app/*.jar"))
qupath_jars.append(os.path.abspath('../../tools/QuPath/lib/app/libopenslide-jni.so'))

# Create an instance of TileStitcher
stitcher = TileStitcher(qupath_jars)


/opt/conda/envs/pathml_standard/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Setting Environment Paths
Java Home is already set
Importing required qupath classes
JVM started successfully


In [2]:
import jpype

In [3]:
jpype.isJVMStarted()

True

### Image Stitching


Now that `TileStitcher` is initialized, we can use it to stitch together tiled images. The run_image_stitching method takes as input a list of TIFF files or a directory containing TIFF files and an output file path.

In [4]:
input_files = glob.glob("path/to/tiles/*.tif")
output_file = "path/to/output.ome.tif"
stitcher.run_image_stitching(input_files, output_file)


This will stitch the tiles together and write the stitched image to the output file in OME-TIFF format.

In [ ]:
# 03:12:56.655 to 04:02:09.477:   1085136,836199,1005036ms to write plane 1,2,3
# Plane 1: 1085136ms is approximately 18 minutes and 5 seconds.
# Plane 2: 836199ms is approximately 13 minutes and 56 seconds.
# Plane 3: 1005036ms is approximately 16 minutes and 45 seconds.

In [4]:
from pathml.preprocessing.tilestitcher import TileStitcher
import glob
import os
# import jpype

In [5]:
qupath_jars = glob.glob(os.path.abspath("../../tools/QuPath/lib/app/*.jar"))
qupath_jars.append(os.path.abspath('../../tools/QuPath/lib/app/libopenslide-jni.so'))

In [6]:
tilestitch = TileStitcher(qupath_jars)

Java path not specified , Setting Java path to /usr/lib/jvm/jdk-17/
JVM was already started


In [ ]:
infile_path= '../../bucket_data/unstitched/MISI3542i_W21-04143_bi016966_M394_OVX_LM/Unmixed Images/'
outfile_path = '../../bucket_data/my_stitched/Pathml_gcp_benchmark_tsdata_3_d18.ome.tif'

In [8]:
import time

start= time.time()
# Run the image stitching process
tilestitch.run_image_stitching(infile_path, outfile_path,[1])
end = time.time()

19:43:47.427 [main] [ERROR] q.l.i.s.o.OpenslideServerBuilder - Could not load OpenSlide native libraries
java.lang.UnsatisfiedLinkError: no openslide-jni in java.library.path: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib
	at java.base/java.lang.ClassLoader.loadLibrary(ClassLoader.java:2429)
	at java.base/java.lang.Runtime.loadLibrary0(Runtime.java:818)
	at java.base/java.lang.System.loadLibrary(System.java:1989)
	at org.openslide.OpenSlideJNI.<clinit>(OpenSlideJNI.java:55)
	at org.openslide.OpenSlide.<clinit>(OpenSlide.java:53)
	at qupath.lib.images.servers.openslide.OpenslideServerBuilder.<clinit>(OpenslideServerBuilder.java:88)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorI

In [ ]:
# 17:36:56.894 to 18:01:51.079 previously

In [9]:
end-start

1104.2805473804474

In [10]:
1104/60

18.4

In [7]:
end-start

1432.6100232601166

In [7]:
import time 
time.time()

1693321509.6048038

In [6]:
end-start

1815.4804677963257

## Benchmarking on Spectral Unmixing data

In [5]:
infile_path= '../../../spectral_unmixing/data/unmixed_data/Unmixed_Images_from_InForm/'
outfile_path = '../../bucket_data/my_stitched/Pathml_gcp_benchmark_sudata_1_d18.ome.tif'

In [6]:
import time

start= time.time()
# Run the image stitching process
tilestitch.run_image_stitching(infile_path, outfile_path,[1,8])
end = time.time()

18:15:34.250 [main] [ERROR] q.l.i.s.o.OpenslideServerBuilder - Could not load OpenSlide native libraries
java.lang.UnsatisfiedLinkError: no openslide-jni in java.library.path: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib
	at java.base/java.lang.ClassLoader.loadLibrary(ClassLoader.java:2429)
	at java.base/java.lang.Runtime.loadLibrary0(Runtime.java:818)
	at java.base/java.lang.System.loadLibrary(System.java:1989)
	at org.openslide.OpenSlideJNI.<clinit>(OpenSlideJNI.java:55)
	at org.openslide.OpenSlide.<clinit>(OpenSlide.java:53)
	at qupath.lib.images.servers.openslide.OpenslideServerBuilder.<clinit>(OpenslideServerBuilder.java:88)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorI

In [8]:
end-start

1009.7496027946472

In [4]:
infile_path= '../../../spectral_unmixing/data/unmixed_data/Unmixed_Images_from_InForm/'
outfile_path = '../../bucket_data/my_stitched/Pathml_gcp_benchmark_sudata_1_d1_only.tif'

In [5]:
import time

start= time.time()
# Run the image stitching process
stitcher.run_image_stitching(infile_path, outfile_path,[1])
end = time.time()

18:37:04.114 [main] [ERROR] q.l.i.s.o.OpenslideServerBuilder - Could not load OpenSlide native libraries
java.lang.UnsatisfiedLinkError: no openslide-jni in java.library.path: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib
	at java.base/java.lang.ClassLoader.loadLibrary(ClassLoader.java:2429)
	at java.base/java.lang.Runtime.loadLibrary0(Runtime.java:818)
	at java.base/java.lang.System.loadLibrary(System.java:1989)
	at org.openslide.OpenSlideJNI.<clinit>(OpenSlideJNI.java:55)
	at org.openslide.OpenSlide.<clinit>(OpenSlide.java:53)
	at qupath.lib.images.servers.openslide.OpenslideServerBuilder.<clinit>(OpenslideServerBuilder.java:88)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorI